# 😊 Sentiment analýza a analýza emocí

**Autor:** Praut s.r.o. - AI Integration & Business Automation

## Co se naučíte:
- Základní a pokročilá sentiment analýza
- Detekce emocí v textu
- Analýza zákaznických recenzí
- Monitoring sociálních sítí

In [ ]:
!pip install -q transformers accelerate torch pandas matplotlib seaborn

In [ ]:
from transformers import pipeline
import torch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

device = 0 if torch.cuda.is_available() else -1
print(f"🖥️ Device: {'GPU' if device == 0 else 'CPU'}")

## 1. Základní sentiment analýza

In [ ]:
# Základní sentiment classifier
sentiment = pipeline("sentiment-analysis", device=device)

texty = [
    "This product exceeded all my expectations!",
    "Terrible experience, complete waste of money.",
    "It's okay, nothing special but does the job.",
    "I'm absolutely in love with this purchase!",
    "Never buying from this company again."
]

print("📊 Základní sentiment analýza:\n")
for text in texty:
    result = sentiment(text)[0]
    emoji = "😊" if result['label'] == 'POSITIVE' else "😞"
    print(f"{emoji} {result['label']} ({result['score']:.1%}): {text}")

## 2. Pokročilá sentiment analýza s více třídami

In [ ]:
# 5-hvězdičkový sentiment (jako recenze)
sentiment_5star = pipeline("sentiment-analysis", 
                           model="nlptown/bert-base-multilingual-uncased-sentiment",
                           device=device)

recenze = [
    "Absolutely amazing product, best purchase ever!",
    "Pretty good, minor issues but overall satisfied.",
    "Average product, nothing to complain about.",
    "Below expectations, not worth the price.",
    "Complete disaster, do not buy this!"
]

print("⭐ 5-hvězdičková analýza:\n")
for text in recenze:
    result = sentiment_5star(text)[0]
    stars = int(result['label'].split()[0])
    print(f"{'⭐' * stars}{'☆' * (5-stars)} ({result['score']:.1%}): {text[:50]}...")

## 3. Analýza emocí

In [ ]:
# Model pro detekci emocí
emotion = pipeline("text-classification", 
                   model="j-hartmann/emotion-english-distilroberta-base",
                   top_k=None,
                   device=device)

texty = [
    "I just got promoted! This is the best day of my life!",
    "I can't believe they cancelled the project after all our hard work.",
    "The presentation is tomorrow and I'm not ready at all.",
    "This constant noise from neighbors is driving me crazy!",
    "Watching old photos brings back so many memories."
]

emoji_map = {
    'joy': '😊', 'sadness': '😢', 'anger': '😠', 
    'fear': '😰', 'surprise': '😲', 'disgust': '🤢', 'neutral': '😐'
}

print("🎭 Analýza emocí:\n")
for text in texty:
    result = emotion(text)[0]
    top_emotion = result[0]
    emoji = emoji_map.get(top_emotion['label'], '❓')
    print(f"{emoji} {top_emotion['label'].upper()} ({top_emotion['score']:.1%})")
    print(f"   \"{text}\"")
    print(f"   Další: {result[1]['label']} ({result[1]['score']:.1%}), {result[2]['label']} ({result[2]['score']:.1%})\n")

## 4. Vícejazyčná sentiment analýza (čeština)

In [ ]:
# Vícejazyčný model
multilang_sentiment = pipeline("sentiment-analysis",
                               model="nlptown/bert-base-multilingual-uncased-sentiment",
                               device=device)

ceske_recenze = [
    "Skvělý produkt, naprostá spokojenost!",
    "Docela dobré, ale čekal jsem víc.",
    "Průměrné, nic extra.",
    "Zklamání, nekvalitní zpracování.",
    "Naprosto hrozné, vyhozené peníze!"
]

print("🇨🇿 České recenze:\n")
for text in ceske_recenze:
    result = multilang_sentiment(text)[0]
    stars = int(result['label'].split()[0])
    print(f"{'⭐' * stars}{'☆' * (5-stars)} {text}")

## 5. Praktická automatizace: Analýza recenzí produktů

In [ ]:
# Simulace dat z e-shopu
recenze_df = pd.DataFrame({
    "produkt": ["Notebook Pro"] * 5 + ["Wireless Mouse"] * 5 + ["USB Hub"] * 5,
    "text": [
        "Amazing laptop, super fast!", "Good but battery life could be better.",
        "Perfect for work and gaming.", "Overpriced for what it offers.",
        "Best laptop I've owned!",
        
        "Works great, comfortable grip.", "Stopped working after 2 weeks.",
        "Good value for money.", "Connection issues with Bluetooth.",
        "Perfect size and weight.",
        
        "Does what it's supposed to do.", "Cheap plastic, feels fragile.",
        "All ports work perfectly.", "Gets hot with heavy use.",
        "Great for the price."
    ],
    "datum": pd.date_range("2024-01-01", periods=15, freq="D")
})

# Analýza všech recenzí
results = sentiment_5star(recenze_df["text"].tolist())

recenze_df["hvezdicky"] = [int(r['label'].split()[0]) for r in results]
recenze_df["sentiment_score"] = [r['score'] for r in results]

print("📊 Přehled recenzí:")
print(recenze_df[["produkt", "hvezdicky", "text"]].to_string(index=False))

In [ ]:
# Agregované statistiky
stats = recenze_df.groupby("produkt").agg({
    "hvezdicky": ["mean", "std", "count"],
    "sentiment_score": "mean"
}).round(2)

stats.columns = ["Průměr ⭐", "Std", "Počet", "Jistota"]
print("\n📈 Statistiky podle produktu:")
print(stats)

In [ ]:
# Vizualizace
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Graf 1: Průměrné hodnocení
produkty = recenze_df.groupby("produkt")["hvezdicky"].mean().sort_values()
colors = ['#ff6b6b' if x < 3 else '#ffd93d' if x < 4 else '#6bcb77' for x in produkty]
axes[0].barh(produkty.index, produkty.values, color=colors)
axes[0].set_xlabel("Průměrné hodnocení ⭐")
axes[0].set_title("Sentiment podle produktu")
axes[0].set_xlim(0, 5)

# Graf 2: Distribuce hodnocení
recenze_df.groupby(["produkt", "hvezdicky"]).size().unstack().plot(
    kind="bar", ax=axes[1], colormap="RdYlGn"
)
axes[1].set_xlabel("Produkt")
axes[1].set_ylabel("Počet recenzí")
axes[1].set_title("Distribuce hodnocení")
axes[1].legend(title="Hvězdičky")

plt.tight_layout()
plt.show()

## 6. Detekce problematických recenzí

In [ ]:
def analyzuj_recenzi(text):
    """Komplexní analýza recenze."""
    
    # Sentiment
    sent_result = sentiment_5star(text)[0]
    hvezdicky = int(sent_result['label'].split()[0])
    
    # Emoce
    emo_result = emotion(text)[0]
    hlavni_emoce = emo_result[0]['label']
    
    # Flagy pro problematické recenze
    flags = []
    if hvezdicky <= 2:
        flags.append("🔴 NEGATIVNÍ")
    if hlavni_emoce == 'anger':
        flags.append("😠 NAŠTVANÝ ZÁKAZNÍK")
    if hlavni_emoce == 'disgust':
        flags.append("🤢 ZNECHUCENÝ")
    
    # Klíčová slova pro eskalaci
    eskalace_slova = ['lawyer', 'sue', 'refund', 'scam', 'fraud', 'report']
    if any(word in text.lower() for word in eskalace_slova):
        flags.append("⚠️ MOŽNÁ ESKALACE")
    
    return {
        'hvezdicky': hvezdicky,
        'emoce': hlavni_emoce,
        'flags': flags,
        'priorita': 'VYSOKÁ' if len(flags) > 1 else 'STŘEDNÍ' if flags else 'NÍZKÁ'
    }

# Test na problematických recenzích
problematicke = [
    "This is a scam! I want my money back immediately or I'll contact my lawyer!",
    "Product works fine, just as described.",
    "Absolutely disgusting quality, worst purchase ever!",
    "Delivery was slow but product is okay."
]

print("🚨 Analýza problematických recenzí:\n")
for text in problematicke:
    vysledek = analyzuj_recenzi(text)
    print(f"📝 \"{text[:60]}...\"")
    print(f"   ⭐ {vysledek['hvezdicky']}/5 | 🎭 {vysledek['emoce']} | Priorita: {vysledek['priorita']}")
    if vysledek['flags']:
        print(f"   Flags: {', '.join(vysledek['flags'])}")
    print()

## 7. Real-time monitoring (simulace)

In [ ]:
import time
from IPython.display import clear_output

# Simulace příchozích recenzí
stream_recenze = [
    "Just received my order, looks great!",
    "WHERE IS MY PACKAGE?! I've been waiting for 3 weeks!",
    "Good quality, fast shipping.",
    "This company is a complete fraud! Reporting to authorities!",
    "Nice product, would recommend."
]

print("📡 Real-time monitoring recenzí (simulace):\n")

statistiky = {'pozitivni': 0, 'negativni': 0, 'eskalace': 0}

for i, recenze in enumerate(stream_recenze):
    vysledek = analyzuj_recenzi(recenze)
    
    # Aktualizace statistik
    if vysledek['hvezdicky'] >= 4:
        statistiky['pozitivni'] += 1
    elif vysledek['hvezdicky'] <= 2:
        statistiky['negativni'] += 1
    if '⚠️ MOŽNÁ ESKALACE' in vysledek.get('flags', []):
        statistiky['eskalace'] += 1
    
    # Výpis
    status = "🔴 ALERT" if vysledek['priorita'] == 'VYSOKÁ' else "🟡" if vysledek['priorita'] == 'STŘEDNÍ' else "🟢"
    print(f"[{i+1}] {status} {recenze[:50]}...")
    print(f"    → {vysledek['hvezdicky']}⭐ | {vysledek['emoce']} | {vysledek['priorita']}")
    
    time.sleep(0.5)  # Simulace delay

print(f"\n📊 Souhrn: ✅ {statistiky['pozitivni']} pozitivních | ❌ {statistiky['negativni']} negativních | ⚠️ {statistiky['eskalace']} eskalací")

---
## 🏁 Shrnutí

- ✅ Základní binární sentiment (pozitivní/negativní)
- ✅ Víceúrovňový sentiment (1-5 hvězdiček)
- ✅ Detekce emocí (radost, smutek, hněv, strach...)
- ✅ Automatická prioritizace recenzí
- ✅ Real-time monitoring

**Další notebook:** Sumarizace a generování textu